# Cybershuttle Molecular Dynamics SDK

This SDK allows users to define, plan, and execute molecular dynamics experiments with ease. In this notebook, we demonstrate how to authenticate, set up a NAMD experiment, add replicas, create an execution plan, and monitor the execution. The experiment object `exp` is an instance of `cybershuttle.base.Experiment`.

In [1]:
import cybershuttle as cs
from cybershuttle import md

## Authenticate for Remote Execution

To authenticate for remote execution, use the `cs.auth.login()` method. This method will prompt you to enter your credentials and authenticate your session.

In [2]:
cs.auth.login()
token = cs.auth.context.access_token

User code: RVLC-SNOJ
Please authenticate by visiting: https://auth.cybershuttle.org/realms/10000000/device?user_code=RVLC-SNOJ
Waiting for authorization...
Authorization successful!


In [3]:
from cybershuttle.airavata import AiravataOperator

assert token is not None

av = AiravataOperator(token)

# Example: get app interface id and storage resource id
app_interface_id = av.get_app_interface_id("NAMD")
storage = av.get_storage()
storage_resource_id = storage.storageResourceId
display(f"application_interface_id={app_interface_id}")
display(f"storage_resource_id={storage_resource_id}")

# Example: upload files
experiment_name = "NAMD_MD_SDK"
app_name = "NAMD"
hpc_name = "login.expanse.sdsc.edu"

experiment = av.launch_experiment(
    experiment_name=experiment_name,
    app_name=app_name,
    computation_resource_name=hpc_name,
    input_path="./data",
)

using legacy validation callback
/Users/yasith/.mamba/envs/cybershuttle-exp/lib/python3.12/site-packages/thrift/transport/TSSLSocket.py:53: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated
  self._context = ssl.SSLContext(ssl_version)


'application_interface_id=NAMD_51d31a65-3526-4ca9-9bd1-d6b6b3e46f75'

'storage_resource_id=js-iguide-cybershuttle.che070035.projects.jetstream-cloud.org_9c15d8af-3d36-4c3c-a07a-0f3b4bb5b903'

100%|██████████| 9/9 [00:07<00:00,  1.24it/s, 📁 par_all36m_prot.prm]


['airavata-dp://c416730f-2df8-417a-a18b-52ece79fc6e7', 'airavata-dp://ec071e64-19d3-413c-b5b9-aac7751875b6', 'airavata-dp://5f8183ff-003d-4b12-9ee7-41ba06f3f3e7', 'airavata-dp://4161f276-fc86-43b6-bf4c-6a35c1611caf', 'airavata-dp://2982626b-d361-4655-a097-4abc0b6ad8e3', 'airavata-dp://44e9e5bc-2aa6-469f-9101-dd25535e8253', 'airavata-dp://8045b1e5-951d-4477-9450-57df1e44f82f', 'airavata-dp://c8c401fd-3f36-4c91-b563-62e7e7931f8e', 'airavata-dp://d9b53b03-9ab8-4b93-a565-02fdbbe4c26f']
{}


## Define a NAMD Experiment

To define a NAMD experiment, we initialize an instance of the `cybershuttle.base.Experiment` class with the required parameters such as the name of the experiment, and the input PDB and PSF files. The experiment object `exp` is created as shown below:

In [ ]:
exp = md.NAMD.initialize(
    name="yasith_namd_experiment",
    config_file="data/pull.conf",
    pdb_file="data/structure.pdb",
    psf_file="data/structure.psf",
    ffp_files=[
      "data/par_all36_water.prm",
      "data/par_all36_prot.prm",
    ],
    other_files=[
      "data/b4pull.pdb",
      "data/b4pull.restart.coor",
      "data/b4pull.restart.vel",
      "data/b4pull.restart.xsc",
    ]
)

## Add Replicas for NAMD Experiment

To add replicas to the NAMD experiment, we use the `add_replica()` method of the `exp` object. This method adds a replica to the experiment. In this example, we add 4 replicas as shown below:

In [ ]:
runtimes = cs.runtime.query()

# always 0th index is local runtime inside
# 0th runtime requires docker to be installed

display(runtimes)

In [ ]:
# choose between any replicas user has access to
# exp.add_replica(*runtimes)

# OR choose any replica that is not local
exp.add_replica(*runtimes[1:])

# OR use only a specific replica
# exp.add_replica(runtimes[1])

## Create Execution Plan

To create an execution plan for the experiment, use the `plan()` method of the `exp` object. This method will generate a plan for the experiment, which can then be described using the `describe()` method. The code to create and describe the plan is shown below:

In [ ]:
plan = exp.plan()  # this will create a plan for the experiment
plan.describe()  # this will describe the plan

In [ ]:
# can also save the plan to a file, or load a plan from a file
plan.save_json("plan.json")
plan = cs.plan.Plan.load_json("plan.json")
plan.describe()

## Execute the Plan

In [ ]:
plan.run()

## Option A - Wait for Completion

In [ ]:
plan.join()

## Option B - Terminate Execution

In [ ]:
plan.stop()

## Option C - Monitor Files During Execution

Displaying the status and files generated by each replica (task)

In [ ]:
for task in plan.tasks:
    status = task.status()
    files = task.files()
    print(status, files)

Displaying the intermediate results generated by each replica (task)

In [ ]:
from matplotlib import pyplot as plt
import pandas as pd

for index, task in enumerate(plan.tasks):

    @cs.task_context(task)
    def visualize():
        data = pd.read_csv("data.csv")
        plt.figure(figsize=(8, 6))
        plt.plot(data["x"], data["y"], marker="o", linestyle="-", linewidth=2, markersize=6)
        plt.title(f"Plot for Replica {index} of {len(plan.tasks)}")

    visualize()